In [ ]:
import xarray as xr
import cfgrib
%pylab inline
from matplotlib.cm import get_cmap
import matplotlib.ticker as mticker
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
import warnings
warnings.filterwarnings('ignore')
from wrf import smooth2d, getvar
import os
import h5py
from netCDF4 import Dataset
import matplotlib.transforms as mtransforms

def get_slp(filepath, min_lat=-90, max_lat=90, min_lon=-180, max_lon=180):
    
    file = xr.open_dataset(filepath, engine='cfgrib',
                backend_kwargs={'filter_by_keys': {'typeOfLevel': 'meanSea','shortName': 'prmsl'}})
    var = file.get('prmsl').to_dataframe()
    latitudes = var.index.get_level_values('latitude')
    longitudes = var.index.get_level_values('longitude')
    map_function = lambda lon: (lon - 360) if (lon > 180) else lon
    remapped_longitudes = longitudes.map(map_function)
    var['longitude'] = remapped_longitudes
    var['latitude'] = latitudes
    lat_filter = (var['latitude'] >= min_lat) & (var['latitude'] <= max_lat)
    lon_filter = (var['longitude'] >= min_lon) & (var['longitude'] <= max_lon)
    var = var.loc[lat_filter & lon_filter]
    var = var.set_index(['latitude', 'longitude']).to_xarray()
    
    return var


In [ ]:
min_lat = 32
max_lat = 48
min_lon = -82
max_lon = -63

slp1_a = get_slp('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111600.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
slp1_f = get_slp('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f060.grib2',
                 min_lat, max_lat, min_lon, max_lon)

slp2_a = get_slp('/p/work1/lloveras/real/nov2018/gfs_files/analysis/gfs.0p25.2018111612.f000.grib2',
                 min_lat, max_lat, min_lon, max_lon)
slp2_f = get_slp('/p/work1/lloveras/real/nov2018/gfs_files/forecast/gfs.0p25.2018111312.f072.grib2',
                 min_lat, max_lat, min_lon, max_lon)


In [ ]:
fig, axd = plt.subplot_mosaic([['left','right'],['cbar','cbar']],
                              constrained_layout=True, figsize=(7.2,3.8), dpi=2000, 
                              gridspec_kw={'width_ratios':[1,1],'height_ratios':[1,0.05]},
                              per_subplot_kw={'left':{'projection':crs.PlateCarree()},
                                              'right':{'projection':crs.PlateCarree()}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### LEFT
#############

cs1 = axd['left'].contour(slp1_a['longitude'], slp1_a['latitude'], slp1_a['prmsl']/100.,
                  levels=np.arange(0,1500,4),
                  colors='k', transform=crs.PlateCarree())
axd['left'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,4),fontsize=6,colors='k')
cs2 = axd['left'].contour(slp1_f['longitude'], slp1_f['latitude'], slp1_f['prmsl']/100.,
                  levels=np.arange(0,1500,4),
                  colors='magenta', transform=crs.PlateCarree())
axd['left'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,4),fontsize=6,colors='magenta')
im1 = axd['left'].contourf(slp1_a['longitude'], slp1_a['latitude'], (slp1_f['prmsl'] - slp1_a['prmsl'])/100.,
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*1.25,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['left'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['left'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['left'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=True
gls.xlocator = mticker.FixedLocator([-80, -75, -70, -65])
gls.ylocator = mticker.FixedLocator([30, 35, 40, 45])

axd['left'].set_title('0000 UTC 16 Nov 2018 ($t$ = 60 h)')
axd['left'].text(0.0, 1.0, '(a)',transform=axd['left'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### RIGHT
#############

cs1 = axd['right'].contour(slp2_a['longitude'], slp2_a['latitude'], slp2_a['prmsl']/100.,
                  levels=np.arange(0,1500,4),
                  colors='k', transform=crs.PlateCarree())
axd['right'].clabel(cs1,fmt='%1.0f',inline=1,levels=np.arange(0,1500,4),fontsize=6,colors='k')
cs2 = axd['right'].contour(slp2_f['longitude'], slp2_f['latitude'], slp2_f['prmsl']/100.,
                  levels=np.arange(0,1500,4),
                  colors='magenta', transform=crs.PlateCarree())
axd['right'].clabel(cs2,fmt='%1.0f',inline=1,levels=np.arange(0,1500,4),fontsize=6,colors='magenta')
im1 = axd['right'].contourf(slp2_a['longitude'], slp2_a['latitude'], (slp2_f['prmsl'] - slp2_a['prmsl'])/100.,
                  levels=np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])*1.25,extend='both',
                  cmap=get_cmap('RdBu_r'), transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['right'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['right'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Add the gridlines
gls = axd['right'].gridlines(draw_labels=True, dms=True,
                   x_inline=False, y_inline=False,linestyle='dashed')
gls.top_labels=False
gls.bottom_labels=True
gls.right_labels=False
gls.left_labels=False
gls.xlocator = mticker.FixedLocator([-80, -75, -70, -65])
gls.ylocator = mticker.FixedLocator([30, 35, 40, 45])

axd['right'].set_title('1200 UTC 16 Nov 2018 ($t$ = 72 h)')
axd['right'].text(0.0, 1.0, '(b)',transform=axd['right'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='horizontal', cax=axd['cbar'])

plt.savefig('/p/work1/lloveras/real/nov2018/figs_nov2018/gfs_slp.pdf',bbox_inches='tight')
plt.show()
